In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import requests
from bs4 import BeautifulSoup

import pandas as pd

import openpyxl
import os
import sys
import time
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from geopy.geocoders import Nominatim

In [ ]:
def kakao(key) :
    
    driver = webdriver.Chrome("chromedriver")
    url = "https://map.kakao.com/"
    driver.get(url)

    searchbox = driver.find_element(By.ID, 'search.keyword.query')
    searchbox.send_keys("서울시 {} 음식점".format(key))

    searchbutton = driver.find_element(By.ID, 'search.keyword.submit')
    driver.execute_script('arguments[0].click();', searchbutton)
    time.sleep(2)

    more = driver.find_element(By.ID, 'info.search.place.more')
    driver.execute_script('arguments[0].click();', more)
    time.sleep(2)

    kakao_df = pd.DataFrame()
    
    for j in range(8) :
    
        title = []
        category = []
        rating = []
        rating_num = []
        review = []
        address = []

        for i in range(5) :

            page = driver.find_element(By.ID, 'info.search.page.no{}'.format(i+1))
            driver.execute_script('arguments[0].click();', page)
            time.sleep(2)
            title_element = "ul.placelist>li.PlaceItem.clickArea>div.head_item.clickArea>strong.tit_name"
            cate_element = "ul.placelist>li.PlaceItem.clickArea>div.head_item.clickArea>span.subcategory.clickable"
            rating_element = "ul.placelist>li.PlaceItem.clickArea>div.rating.clickArea>span.score>em.num"
            ratingnum_element = "ul.placelist>li.PlaceItem.clickArea>div.rating.clickArea>span.score>a.numberofscore"
            re_element = "ul.placelist>li.PlaceItem.clickArea>div.rating.clickArea>a.review>em"
            add_element = "ul.placelist>li.PlaceItem.clickArea>div.info_item>div.addr"

            for x in driver.find_elements(By.CSS_SELECTOR, title_element) :
                title.append(x.text)

            for x in driver.find_elements(By.CSS_SELECTOR, cate_element) :
                category.append(x.text)

            for x in driver.find_elements(By.CSS_SELECTOR, rating_element) :
                rating.append(x.text)    

            for x in driver.find_elements(By.CSS_SELECTOR, ratingnum_element) :
                rating_num.append(x.text)

            for x in driver.find_elements(By.CSS_SELECTOR, re_element) :
                review.append(x.text)

            for x in driver.find_elements(By.CSS_SELECTOR, add_element) :
                address.append(x.text)

            df0 = pd.DataFrame({'Store' : title, 'Category' : category, 
                               'Rating' : rating, 'RatingNum' : rating_num, 'Review' : review, 'Address' : address})
        
        kakao_df = pd.concat([kakao_df, df0], axis = 0)

        nxt = driver.find_element(By.ID, 'info.search.page.next')
        driver.execute_script('arguments[0].click();', nxt)
        time.sleep(2)
    
    driver.close() 

    return kakao_df


# 좌표변환
def geocoding(area):
    try:
        geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
        geo = geolocoder.geocode(area)
        x_y = [geo.latitude, geo.longitude]
        return x_y
    
    except:
        return [0,0]

    
# 전처리
def preprocessing(df) :
    
    new = []
    for t in df['Store'] :
        new.append(' '.join(t.split(' ')[1:]))
    df['newTitle'] = new
    
    new_add1 = []
    new_add2 = []
    
    for x in df['Address'] :
        a = x.split('\n(지번)')
        new_add1.append(a[0])
        new_add2.append(b[1])
        
    df['도로명'] = new_add1
    df['지번'] = new_add2
    
    
    lati = []
    longit = []
    for y in df['도로명'] :
        y = ' '.join(y.split(' ')[:4])
        
        lat = geocoding(y)[0]
        long = geocoding(y)[1]
        
        lati.append(lat)
        longit.append(long)
    
    df['latitude'] = lati
    df['longitude'] = longit
    
    return df


In [ ]:
key_lst = ['노원구', '중랑구', '광진구', '도봉구', '강북구', '동대문구', '성동구', '성북구',
           '종로구', '중구', '용산구', '은평구', '서대문구', '마포구' ,'강서구', '양천구', '구로구', 
           '영등포구', '금천구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']

final_df = pd.DataFrame()

for key in key_lst :
    df = kakao(key)
    final_df = pd.concat([final_df, df], axis = 0)
    
final_df.shape

# 카카오맵 리뷰 데이터 수집

### 1. '서울시 음식점' 상위 10개 음식점 가게명 수집

In [78]:
def kakao() :
    
    driver = webdriver.Chrome("chromedriver")
    url = "https://map.kakao.com/"
    driver.get(url)

    searchbox = driver.find_element(By.ID, 'search.keyword.query')
    searchbox.send_keys("서울시 음식점")

    searchbutton = driver.find_element(By.ID, 'search.keyword.submit')
    driver.execute_script('arguments[0].click();', searchbutton)
    time.sleep(2)
    
    sort = driver.find_element(By.XPATH, '//*[@id="info.search.place.sort"]/li[2]')
    driver.execute_script('arguments[0].click();', sort)
    time.sleep(1.5)

    more = driver.find_element(By.ID, 'info.search.place.more')
    driver.execute_script('arguments[0].click();', more)
    time.sleep(2)

    kakao_df = pd.DataFrame()
    
    for j in range(3) :
    
        title = []
        category = []
        rating = []
        rating_num = []
        review = []
        address = []

        for i in range(2) :

            page = driver.find_element(By.ID, 'info.search.page.no{}'.format(i+1))
            driver.execute_script('arguments[0].click();', page)
            time.sleep(2)
            title_element = "ul.placelist>li.PlaceItem.clickArea>div.head_item.clickArea>strong.tit_name"
            cate_element = "ul.placelist>li.PlaceItem.clickArea>div.head_item.clickArea>span.subcategory.clickable"
            rating_element = "ul.placelist>li.PlaceItem.clickArea>div.rating.clickArea>span.score>em.num"
            ratingnum_element = "ul.placelist>li.PlaceItem.clickArea>div.rating.clickArea>span.score>a.numberofscore"
            re_element = "ul.placelist>li.PlaceItem.clickArea>div.rating.clickArea>a.review>em"
            add_element = "ul.placelist>li.PlaceItem.clickArea>div.info_item>div.addr"

            for x in driver.find_elements(By.CSS_SELECTOR, title_element) :
                title.append(x.text)

            for x in driver.find_elements(By.CSS_SELECTOR, cate_element) :
                category.append(x.text)
                
            for x in driver.find_elements(By.CSS_SELECTOR, rating_element) :
                rating.append(x.text)    

            for x in driver.find_elements(By.CSS_SELECTOR, ratingnum_element) :
                rating_num.append(x.text)

            for x in driver.find_elements(By.CSS_SELECTOR, re_element) :
                review.append(x.text)

            for x in driver.find_elements(By.CSS_SELECTOR, add_element) :
                address.append(x.text)                

            df0 = pd.DataFrame({'Store' : title, 'Category' : category, 
                               'Rating' : rating, 'RatingNum' : rating_num, 'Review' : review, 'Address' : address})
        
        kakao_df = pd.concat([kakao_df, df0], axis = 0)

        nxt = driver.find_element(By.ID, 'info.search.page.next')
        driver.execute_script('arguments[0].click();', nxt)
        time.sleep(2)
    
    driver.close() 

    return kakao_df

def preprocess(df) :
    df0 = df.copy()
    lst = []
    for x in df['Store'] :
        lst.append(' '.join(x.split(' ')[1:]))
    
    df0['Store'] = lst
    
    re = []
    for x in df['Review'] :
        re.append(int(x.replace(',', '')))
    df0['Review'] = re
    
    return df0

In [79]:
df = kakao()
df0 = preprocess(df)

In [88]:
df0 = df0.reset_index(drop = True)

In [92]:
def review_text(key, scroll):
    driver = webdriver.Chrome("chromedriver")
    url = "https://map.kakao.com/"
    driver.get(url)

    searchbox = driver.find_element(By.ID, 'search.keyword.query')
    searchbox.send_keys('서울 {}'.format(key))


    searchbutton = driver.find_element(By.ID, 'search.keyword.submit')
    driver.execute_script('arguments[0].click();', searchbutton)
    time.sleep(2)

    new_path = '//*[@id="info.search.place.list"]/li/div[4]/a'
    new_window = driver.find_element(By.XPATH, new_path)
    driver.execute_script('arguments[0].click();', new_window)
    time.sleep(2)

    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(3)

    for i in range(scroll) :
        more = driver.find_elements(By.CSS_SELECTOR, 'span.txt_more')[0]
        driver.execute_script('arguments[0].click();', more)
        time.sleep(1)

    review = driver.find_elements(By.CSS_SELECTOR, 'p.txt_comment.txt_fold')


    re_lst = []
    for x in review :
        re_lst.append(x.text)

    time.sleep(2)
    driver.quit()
    
    return re_lst

In [93]:

# store = list(df0['Store'])
# scroll = 
# review_lst = []
# for key in store :
#     review = review_text(key)
#     review_lst.append(review)

review_lst = []

for i in range(len(df0)) :
    key = df0.loc[i, 'Store']
    scroll = (df0.loc[i, 'Review'] - 3) // 5
    
    review = review_text(key, scroll)
    review_lst.append(review)
    
key = df0.loc[1, 'Store']
scroll = (df0.loc[1, 'Review'] - 3) // 5

review = review_text(key, scroll)
review_lst.append(review)
    

In [94]:
review_lst

[['강남의 랭면 행복,,, 다른 평냉보다,,\n간이 좀 짠 특색,, 고기류도 근본,,,\n\n행복은,,, 어디에나 있는 것,,, ^^  \n@azossii\n더보기',
  '육수에 간이 강해서 살짝 놀랐다가 먹다보니 또 적응됨. 그날따라 내 혀에 무슨 일이 있었나 면 메밀향이랑 육수 육향이 잘 느껴지지 않아 괴로왔는데 냉면 안에 편육 두 조각이 천국이었음; 육수는 뭔가 잘 모르겠음..ㅜ\n\n그치만 역시 내 마음 속 원픽은 역시 “필동”인 건가…\n필동>을밀대/을지>…. 우래옥.. 그 다음 진미..\n후… 필동 마렵다…\n\n너무 시끄러워서 다시 가지는 않을듯\n더보기',
  'Honeypig in Seoul\n\n솔직한 나의 생각:\n\n- 개인적인 취향저격한 슴슴하지만 육수향이 약간 나서 중독성 있는 적당함\n- 개취임 !!\n\n재방문 의사 :\n☆☆☆☆☆ 85 - 100 %\n☆☆☆☆ 70 - 84 %\n☆☆☆ 50 - 69 %\n☆☆ 20 - 49 %\n☆ 10 % 이하\n\nBy @travelling_honeypig\n더보기',
  "ㅡㅡ22.09.\n항공 데시벨을 뚫고 먹고 옴(선주후면)\n맛 예전과 同\n\n만두 추가- 평이 담박泊 호평\n\n\nㅡㅡ\n22.03.경\n\n2인 어복쟁반 大.\n굳.\n\n*팁: 붐빌 때 자리안내 없으면 걍 별관 들가면 됨\n\n\n\n육유(肉油)에 절여진 쑥갓의 맛. \n'좋은 기억'\n\n\n\nㅡㅡㅡ\n22.연초\n\n육수 초 상등 '여전(如前)'. \n면발 리스크 有.\n\n\n\n\nㅡㅡㅡ\n21.하반기\n\n-'수인가능성 측정기'\n평냉 처음먹는 검증대상을 불러내, 싱겁다며 연신 불평하면 바로 차단.\n\n-슬램덩크로 치면 정우성.\n\n-일반음식점 별3개 만점 '무시'.\n\n-'하늘은 을지면옥을 낳고 왜 또 진미를 낳았는가'\n생각해 볼 일. (삼국연의 주유 대사 오마쥬)\n\n*비고\n주차요원의 불친절은 웨이팅을 줄이는 추진력을 얻기 위한 포석(布石).\n더보기",
  '그냥 평냉맛, 평냉중에는 

In [60]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import re

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from nltk.tokenize import word_tokenize, WordPunctTokenizer
from konlpy.tag import Okt, Mecab, Kkma

from wordcloud import WordCloud
from collections import Counter
from PIL import Image
from eunjeon import Mecab

In [40]:
text = ' '.join(re_lst)
mecab = Mecab()
text = re.compile('[^가-힣\s]').sub('', text)
pos = mecab.nouns(text)
pos
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()

pos = [x for x in pos if x not in stopwords]
pos

['여기',
 '살',
 '존',
 '맛',
 '탱',
 '사이즈',
 '바선',
 '사건',
 '사실',
 '뒤',
 '위생',
 '년차',
 '때',
 '번',
 '칼',
 '개',
 '개',
 '칼',
 '개',
 '교환',
 '닭',
 '베이스',
 '육',
 '순데',
 '여기',
 '바지락',
 '칼국수',
 '불친절',
 '건',
 '소린지',
 '거기',
 '검정',
 '색',
 '옷',
 '분',
 '데',
 '때',
 '껌',
 '하나',
 '불친절',
 '거',
 '소리',
 '것',
 '진작',
 '예전',
 '구경',
 '때',
 '곳',
 '십',
 '년',
 '만',
 '거',
 '예전',
 '맛',
 '칼국수',
 '식감',
 '국물',
 '간고기',
 '마늘',
 '향',
 '맛',
 '짜쿵',
 '마늘',
 '것',
 '절',
 '김치',
 '맛',
 '년',
 '현재',
 '맛',
 '이전',
 '코로나',
 '시국',
 '장사',
 '이전',
 '테이블',
 '아크릴',
 '칸막이',
 '팀',
 '합석',
 '팔',
 '불편',
 '앞',
 '번잡',
 '날',
 '이용']

In [54]:
df0 = pd.DataFrame(pd.DataFrame(pos).value_counts(), columns = ['count']).reset_index()
df0

,0,count
0,맛,5
1,때,3
2,개,3
3,예전,2
4,마늘,2
...,...,...
63,사건,1
64,사실,1
65,사이즈,1
66,색,1


In [64]:
sudo apt-get install libfreetype6-dev 

pip uninstall pillow 

pip install pillow==2.8.2

SyntaxError: invalid syntax (1375873632.py, line 1)

In [63]:
df = pd.DataFrame(pd.DataFrame(pos).value_counts(), columns = ['count']).reset_index()
df = df.reset_index().rename(columns = {0 : 'word'})

def color_light(word, font_size, position,orientation,random_state=None, **kwargs):
    return("hsl({:d},{:d}%, {:d}%)".format(np.random.randint(239,313),np.random.randint(26,32),np.random.randint(45,80)))

def color_dark(word, font_size, position,orientation,random_state=None, **kwargs):
    return("hsl({:d},{:d}%, {:d}%)".format(np.random.randint(239,313),np.random.randint(26,32),np.random.randint(10,30)))


fre = dict(zip(df['word'], df['count']))
wc = WordCloud(background_color="white")
gen = wc.generate_from_frequencies(fre)

fig = plt.figure()
plt.imshow(gen)                   
plt.axis('off')

ImportError: The _imagingft C module is not installed

In [58]:
# 크롤링
def kakao(key) :
    
    driver = webdriver.Chrome("chromedriver")
    url = "https://map.kakao.com/"
    driver.get(url)

    searchbox = driver.find_element(By.ID, 'search.keyword.query')
    searchbox.send_keys("서울시 {} 음식점".format(key))

    searchbutton = driver.find_element(By.ID, 'search.keyword.submit')
    driver.execute_script('arguments[0].click();', searchbutton)
    time.sleep(2)

    more = driver.find_element(By.ID, 'info.search.place.more')
    driver.execute_script('arguments[0].click();', more)
    time.sleep(2)

    kakao_df = pd.DataFrame()
    
    for j in range(8) :
    
        title = []
        category = []
        rating = []
        rating_num = []
        review = []
        address = []

        for i in range(5) :

            page = driver.find_element(By.ID, 'info.search.page.no{}'.format(i+1))
            driver.execute_script('arguments[0].click();', page)
            time.sleep(2)
            title_element = "ul.placelist>li.PlaceItem.clickArea>div.head_item.clickArea>strong.tit_name"
            cate_element = "ul.placelist>li.PlaceItem.clickArea>div.head_item.clickArea>span.subcategory.clickable"
            rating_element = "ul.placelist>li.PlaceItem.clickArea>div.rating.clickArea>span.score>em.num"
            ratingnum_element = "ul.placelist>li.PlaceItem.clickArea>div.rating.clickArea>span.score>a.numberofscore"
            re_element = "ul.placelist>li.PlaceItem.clickArea>div.rating.clickArea>a.review>em"
            add_element = "ul.placelist>li.PlaceItem.clickArea>div.info_item>div.addr"

            for x in driver.find_elements(By.CSS_SELECTOR, title_element) :
                title.append(x.text)

            for x in driver.find_elements(By.CSS_SELECTOR, cate_element) :
                category.append(x.text)

            for x in driver.find_elements(By.CSS_SELECTOR, rating_element) :
                rating.append(x.text)    

            for x in driver.find_elements(By.CSS_SELECTOR, ratingnum_element) :
                rating_num.append(x.text)

            for x in driver.find_elements(By.CSS_SELECTOR, re_element) :
                review.append(x.text)

            for x in driver.find_elements(By.CSS_SELECTOR, add_element) :
                address.append(x.text)

            df0 = pd.DataFrame({'Title' : title, 'Category' : category, 
                               'Rating' : rating, 'RatingNum' : rating_num, 'Review' : review, 'Address' : address})
        
        kakao_df = pd.concat([kakao_df, df0], axis = 0)

        nxt = driver.find_element(By.ID, 'info.search.page.next')
        driver.execute_script('arguments[0].click();', nxt)
        time.sleep(2)
    
    driver.close() 

    return kakao_df

# def final_kakaomap(key) :
    
#     driver = webdriver.Chrome("chromedriver")
#     url = "https://map.kakao.com/"
#     driver.get(url)

#     searchbox = driver.find_element(By.ID, 'search.keyword.query')
#     searchbox.send_keys("서울시 {} 음식점".format(key))

#     searchbutton = driver.find_element(By.ID, 'search.keyword.submit')
#     driver.execute_script('arguments[0].click();', searchbutton)
#     time.sleep(2)

#     more = driver.find_element(By.ID, 'info.search.place.more')
#     driver.execute_script('arguments[0].click();', more)
#     time.sleep(2)

#     kakao_df = pd.DataFrame()

#     for j in range(8) :

#         df0 = kakao()

#         kakao_df = pd.concat([kakao_df, df0], axis = 0)

#         nxt = driver.find_element(By.ID, 'info.search.page.next')
#         driver.execute_script('arguments[0].click();', nxt)
#         time.sleep(2)

#     driver.close() 
    
#     return kakao_df

# 주소 좌표 변환
def geocoding(area):
    try:
        geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
        geo = geolocoder.geocode(area)
        x_y = [geo.latitude, geo.longitude]
        return x_y
    
    except:
        return [0,0]

# 전처리
def preprocessing(df) :
    
    new = []
    for t in df['Title'] :
        new.append(' '.join(t.split(' ')[1:]))
    df['newTitle'] = new
    
    new_add1 = []
    new_add2 = []
    
    for x in df['Address'] :
        a = x.split('\n')
        new_add1.append(a[0])
        new_add2.append(b[1])
        
    df['도로명'] = new_add1
    df['지번'] = new_add2
    
    
    lati = []
    longit = []
    for y in df['도로명'] :
        y = ' '.join(y.split(' ')[:4])
        
        lat = geocoding(y)[0]
        long = geocoding(y)[1]
        
        lati.append(lat)
        longit.append(long)
    
    df['latitude'] = lati
    df['longitude'] = longit
    
    return df


In [4]:
# def final_kakaomap(key) :
    
#     driver = webdriver.Chrome("chromedriver")
#     url = "https://map.kakao.com/"
#     driver.get(url)

#     searchbox = driver.find_element(By.ID, 'search.keyword.query')
#     searchbox.send_keys("서울시 {} 음식점".format(key))

#     searchbutton = driver.find_element(By.ID, 'search.keyword.submit')
#     driver.execute_script('arguments[0].click();', searchbutton)
#     time.sleep(2)

#     more = driver.find_element(By.ID, 'info.search.place.more')
#     driver.execute_script('arguments[0].click();', more)
#     time.sleep(2)

#     kakao_df = pd.DataFrame()

#     for j in range(8) :

#         df0 = kakao()

#         kakao_df = pd.concat([kakao_df, df0], axis = 0)

#         nxt = driver.find_element(By.ID, 'info.search.page.next')
#         driver.execute_script('arguments[0].click();', nxt)
#         time.sleep(2)

#     driver.close() 
    
#     return kakao_df

In [17]:
key_lst = ['노원구', '중랑구', '광진구', '도봉구', '강북구', '동대문구', '성동구', '성북구',
           '종로구', '중구', '용산구', '은평구', '서대문구', '마포구' ,'강서구', '양천구', '구로구', 
           '영등포구', '금천구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']

final_df2 = pd.DataFrame()
for key in key_lst :
    df = kakao(key)
    final_df2 = pd.concat([final_df, df], axis = 0)
    
final_df2.shape

(11550, 6)

In [18]:
df = pd.concat([final_df, final_df2], axis = 0)

In [20]:
final_kakao_df = df.drop_duplicates(subset='Title')
final_kakao_df.shape

(10473, 6)

In [80]:
final_kakao_df.drop(['지번'], axis = 1).rename(columns = {'newTitle' : 'Store'})
# final_kakao_df.rename(columns = {'newTitle' : 'Store'})

,Title,Category,Rating,RatingNum,Review,Address,Store,도로명,latitude,longitude
0,A 강강술래 상계지점,"육류,고기",3.4,61건,135,서울 노원구 동일로 1628 1층\n(지번) 상계동 1025-4,강강술래 상계지점,서울 노원구 동일로 1628 1층,37.673692,127.055257
1,B 감동식당,한식,4.9,385건,228,서울 노원구 한글비석로47길 58 1층\n(지번) 상계동 434-49,감동식당,서울 노원구 한글비석로47길 58 1층,37.659783,127.066045
2,C 로니로티 노원점,양식,4.2,47건,250,서울 노원구 상계로 63-7 청우빌딩 2층 201호\n(지번) 상계동 603-8,로니로티 노원점,서울 노원구 상계로 63-7 청우빌딩 2층 201호,37.656717,127.063254
3,D 라라브레드 공릉점,디저트카페,3.9,112건,355,서울 노원구 공릉로41길 32\n(지번) 공릉동 350-24,라라브레드 공릉점,서울 노원구 공릉로41길 32,37.624630,127.078503
4,E 경복식당,한식,4.7,126건,171,서울 노원구 공릉로39길 10\n(지번) 공릉동 740,경복식당,서울 노원구 공릉로39길 10,37.623748,127.078514
...,...,...,...,...,...,...,...,...,...,...
45,A 어사출또 암사역점,회,4.0,8건,14,서울 강동구 올림픽로 794 1층\n(지번) 암사동 500-2,어사출또 암사역점,서울 강동구 올림픽로 794 1층,37.541280,127.124340
46,B 김선장횟집,회,5.0,1건,6,서울 강동구 올림픽로98길 49\n(지번) 암사동 495,김선장횟집,서울 강동구 올림픽로98길 49,37.550800,127.129570
47,C 종로곱창,"곱창,막창",4.5,8건,9,서울 강동구 구천면로34길 38\n(지번) 천호동 201-19,종로곱창,서울 강동구 구천면로34길 38,37.543080,127.134375
48,D 중경해선훠궈,중식,4.8,10건,73,서울 강동구 천호옛길 94-6\n(지번) 성내동 48-12,중경해선훠궈,서울 강동구 천호옛길 94-6,37.534100,127.123770


In [59]:
kakaomap_final = preprocessing(final_kakao_df)

C:\Users\1dk1d\AppData\Local\Temp\ipykernel_20544\1895395143.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['newTitle'] = new
C:\Users\1dk1d\AppData\Local\Temp\ipykernel_20544\1895395143.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['도로명'] = new_add1
C:\Users\1dk1d\AppData\Local\Temp\ipykernel_20544\1895395143.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [64]:
kakaomap_final.to_csv('최최최종_카카오맵서울.csv', encoding = 'cp949')

In [66]:
pd.read_csv('최최최종_카카오맵서울.csv',  encoding = 'cp949')

,Unnamed: 0,Title,Category,Rating,RatingNum,Review,Address,newTitle,도로명,지번,latitude,longitude
0,0,A 강강술래 상계지점,"육류,고기",3.4,61건,135,서울 노원구 동일로 1628 1층\n(지번) 상계동 1025-4,강강술래 상계지점,서울 노원구 동일로 1628 1층,지,37.673692,127.055257
1,1,B 감동식당,한식,4.9,385건,228,서울 노원구 한글비석로47길 58 1층\n(지번) 상계동 434-49,감동식당,서울 노원구 한글비석로47길 58 1층,지,37.659783,127.066045
2,2,C 로니로티 노원점,양식,4.2,47건,250,서울 노원구 상계로 63-7 청우빌딩 2층 201호\n(지번) 상계동 603-8,로니로티 노원점,서울 노원구 상계로 63-7 청우빌딩 2층 201호,지,37.656717,127.063254
3,3,D 라라브레드 공릉점,디저트카페,3.9,112건,355,서울 노원구 공릉로41길 32\n(지번) 공릉동 350-24,라라브레드 공릉점,서울 노원구 공릉로41길 32,지,37.624630,127.078503
4,4,E 경복식당,한식,4.7,126건,171,서울 노원구 공릉로39길 10\n(지번) 공릉동 740,경복식당,서울 노원구 공릉로39길 10,지,37.623748,127.078514
...,...,...,...,...,...,...,...,...,...,...,...,...
10468,45,A 어사출또 암사역점,회,4.0,8건,14,서울 강동구 올림픽로 794 1층\n(지번) 암사동 500-2,어사출또 암사역점,서울 강동구 올림픽로 794 1층,지,37.541280,127.124340
10469,46,B 김선장횟집,회,5.0,1건,6,서울 강동구 올림픽로98길 49\n(지번) 암사동 495,김선장횟집,서울 강동구 올림픽로98길 49,지,37.550800,127.129570
10470,47,C 종로곱창,"곱창,막창",4.5,8건,9,서울 강동구 구천면로34길 38\n(지번) 천호동 201-19,종로곱창,서울 강동구 구천면로34길 38,지,37.543080,127.134375
10471,48,D 중경해선훠궈,중식,4.8,10건,73,서울 강동구 천호옛길 94-6\n(지번) 성내동 48-12,중경해선훠궈,서울 강동구 천호옛길 94-6,지,37.534100,127.123770


In [21]:
df.shape

(22550, 6)

In [12]:
final_df.shape

(11000, 6)

In [15]:
final_df.to_csv('동작구까지.csv')

In [16]:
pd.read_csv('동작구까지.csv')

,Unnamed: 0,Title,Category,Rating,RatingNum,Review,Address
0,0,A 강강술래 상계지점,"육류,고기",3.4,61건,135,서울 노원구 동일로 1628 1층\n(지번) 상계동 1025-4
1,1,B 감동식당,한식,4.9,385건,228,서울 노원구 한글비석로47길 58 1층\n(지번) 상계동 434-49
2,2,C 로니로티 노원점,양식,4.2,47건,250,서울 노원구 상계로 63-7 청우빌딩 2층 201호\n(지번) 상계동 603-8
3,3,D 라라브레드 공릉점,디저트카페,3.9,112건,355,서울 노원구 공릉로41길 32\n(지번) 공릉동 350-24
4,4,E 경복식당,한식,4.7,126건,171,서울 노원구 공릉로39길 10\n(지번) 공릉동 740
...,...,...,...,...,...,...,...
10995,45,A 미각양꼬치,양꼬치,2.7,15건,3,"서울 동작구 만양로 94-1 1,2층\n(지번) 노량진동 119-156"
10996,46,B 천안문,중국요리,4.7,12건,4,서울 동작구 보라매로 65\n(지번) 신대방동 344-41
10997,47,C 광명수산,회,2.7,3건,7,서울 동작구 동작대로29가길 5\n(지번) 사당동 89-61
10998,48,D 불타는소금구이 상도동점,"육류,고기",4.2,12건,28,서울 동작구 상도로58길 2\n(지번) 상도동 490-5


In [4]:
kakao_df_0 = kakao_df[:500]
kakao_df_0.tail()

,Title,Category,Rating,RatingNum,Review,Address
45,A 됐소 강남점,"육류,고기",4.3,10건,93,"서울 서초구 서초대로78길 46 익산빌딩 2,3층\n(지번) 서초동 1330-15"
46,B 등촌칼국수,국수,3.8,158건,183,서울 강서구 화곡로64길 68 1~3층\n(지번) 등촌동 654-95
47,C 루엘드파리,"제과,베이커리",3.9,143건,330,"서울 서초구 서초중앙로 18 1층 107,112,117,118호\n(지번) 서초동 ..."
48,D 대려도,중국요리,3.6,37건,50,서울 강남구 역삼로 118 2층\n(지번) 역삼동 833
49,E 영화루,중국요리,3.1,99건,236,서울 종로구 자하문로7길 65\n(지번) 누하동 25-1


In [5]:
kakao_df.shape, kakao_df_0.shape

((550, 6), (500, 6))

In [6]:
test = kakao_df.drop_duplicates(subset = 'Title')
test.shape

(500, 6)

In [7]:
test.to_csv('kakaomap_서울.csv')

In [9]:
test.tail()

,Title,Category,Rating,RatingNum,Review,Address
45,A 됐소 강남점,"육류,고기",4.3,10건,93,"서울 서초구 서초대로78길 46 익산빌딩 2,3층\n(지번) 서초동 1330-15"
46,B 등촌칼국수,국수,3.8,158건,183,서울 강서구 화곡로64길 68 1~3층\n(지번) 등촌동 654-95
47,C 루엘드파리,"제과,베이커리",3.9,143건,330,"서울 서초구 서초중앙로 18 1층 107,112,117,118호\n(지번) 서초동 ..."
48,D 대려도,중국요리,3.6,37건,50,서울 강남구 역삼로 118 2층\n(지번) 역삼동 833
49,E 영화루,중국요리,3.1,99건,236,서울 종로구 자하문로7길 65\n(지번) 누하동 25-1


In [10]:
final_kakao = preprocessing(test)

C:\Users\1dk1d\AppData\Local\Temp\ipykernel_11864\1681530135.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['newTitle'] = new
C:\Users\1dk1d\AppData\Local\Temp\ipykernel_11864\1681530135.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['도로명'] = new_add1
C:\Users\1dk1d\AppData\Local\Temp\ipykernel_11864\1681530135.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [16]:
final_kakao = final_kakao.reset_index(drop = True)
final_kakao[final_kakao['latitude'] == 0]

In [36]:
final_kakao.loc[240, ['latitude', 'longitude']] = geocoding('서울 영등포구 대림동 963-9')
final_kakao.loc[258, ['latitude', 'longitude']] = geocoding('서울 중구 명동1가 59-1')

final_kakao.loc[[240, 258]]

In [43]:
final_kakao.isnull().sum()

Title        0
Category     0
Rating       0
RatingNum    0
Review       0
Address      0
newTitle     0
도로명          0
지번           0
latitude     0
longitude    0
dtype: int64

In [44]:
final_kakao.to_csv('FinalKakaoMap_서울.csv')

In [45]:
kakaomap = final_kakao[['newTitle', 'Category', 'Rating', 'Review', '도로명', '지번', 'latitude', 'longitude']]

In [48]:
kakaomap.to_csv('test.csv')

In [49]:
pd.read_csv('test.csv')

,Unnamed: 0,newTitle,Category,Rating,Review,도로명,지번,latitude,longitude
0,0,명동교자 본점,분식,3.6,"2,201",서울 중구 명동10길 29,명동2가 25-36,37.563189,126.985590
1,1,진미평양냉면,냉면,3.6,819,서울 강남구 학동로 305-3 정각빌딩 1층,논현동 115-10,37.513421,127.028583
2,2,금돼지식당,"육류,고기",3.8,"1,095",서울 중구 다산로 149,신당동 370-69,37.550779,127.007644
3,3,어니언 안국점,카페,3.5,"1,432",서울 종로구 계동길 5,계동 146-1,37.578268,126.986744
4,4,중앙해장,해장국,3.9,943,서울 강남구 영동대로86길 17 육인빌딩 1층,대치동 996-16,37.508293,127.065495
...,...,...,...,...,...,...,...,...,...
495,495,됐소 강남점,"육류,고기",4.3,93,"서울 서초구 서초대로78길 46 익산빌딩 2,3층",서초동 1330-15,37.496167,127.027668
496,496,등촌칼국수,국수,3.8,183,서울 강서구 화곡로64길 68 1~3층,등촌동 654-95,37.556247,126.856495
497,497,루엘드파리,"제과,베이커리",3.9,330,"서울 서초구 서초중앙로 18 1층 107,112,117,118호",서초동 1445-13,37.484000,127.017323
498,498,대려도,중국요리,3.6,50,서울 강남구 역삼로 118 2층,역삼동 833,37.497199,127.041991


In [70]:
df = pd.read_csv('최최최종_카카오맵서울.csv', encoding = 'cp949').drop(['지번', 'Address', 'Title', 'Unnamed: 0'], axis = 1)
df
df.to_csv('최종_카카오맵_서울.csv', encoding = 'cp949')

In [77]:
lst = []
for x in df['도로명'] :
    y = x.split(' ')
    lst.append(y[1])
df['행정구'] = lst
df.to_csv('test_행정구.csv', encoding = 'cp949')

In [75]:
df['행정구'].value_counts()

중구      501
노원구     500
광진구     500
동대문구    500
중랑구     500
강서구     499
강북구     499
성동구     499
성북구     499
종로구     499
서대문구    499
마포구     499
구로구     498
양천구     498
은평구     498
용산구     498
도봉구     498
영등포구    497
동작구     497
강동구     497
금천구     496
관악구       2
Name: 행정구, dtype: int64

In [38]:
driver = webdriver.Chrome("chromedriver")
url = "https://map.kakao.com/"
driver.get(url)

searchbox = driver.find_element(By.ID, 'search.keyword.query')
searchbox.send_keys("서울시 음식점")

searchbutton = driver.find_element(By.ID, 'search.keyword.submit')
driver.execute_script('arguments[0].click();', searchbutton)
time.sleep(2)

more = driver.find_element(By.ID, 'info.search.place.more')
driver.execute_script('arguments[0].click();', more)
time.sleep(2)

data_lst = []

for j in range(7) :
#     for i in range(5) :

#         page = driver.find_element(By.ID, 'info.search.page.no{}'.format(i+1))
#         driver.execute_script('arguments[0].click();', page)
#         time.sleep(2)

# #         data = driver.find_elements(By.CSS_SELECTOR, "ul.placelist>li.PlaceItem.clickArea")

#         data = driver.find_elements(By.CSS_SELECTOR, "ul.placelist>li.PlaceItem.clickArea>div.head_item.clickArea>strong.tit_name")
#         for x in data :
#             data_lst.append(x.text)
    df0 = kakao()

    nxt = driver.find_element(By.ID, 'info.search.page.next')
    driver.execute_script('arguments[0].click();', nxt)
    time.sleep(2)


driver.close() 


In [39]:
len(data_lst)


500

In [22]:
driver = webdriver.Chrome("chromedriver")
url = "https://map.kakao.com/"
driver.get(url)

searchbox = driver.find_element(By.ID, 'search.keyword.query')
searchbox.send_keys("서울시 음식점")

searchbutton = driver.find_element(By.ID, 'search.keyword.submit')
driver.execute_script('arguments[0].click();', searchbutton)
time.sleep(2)

more = driver.find_element(By.ID, 'info.search.place.more')
driver.execute_script('arguments[0].click();', more)
time.sleep(2)

title = []
category = []
rating = []
rating_num = []
review = []
address = []

for i in range(5) :

    page = driver.find_element(By.ID, 'info.search.page.no{}'.format(i+1))
    driver.execute_script('arguments[0].click();', page)
    time.sleep(2)
    title_element = "ul.placelist>li.PlaceItem.clickArea>div.head_item.clickArea>strong.tit_name"
    cate_element = "ul.placelist>li.PlaceItem.clickArea>div.head_item.clickArea>span.subcategory.clickable"
    rating_element = "ul.placelist>li.PlaceItem.clickArea>div.rating.clickArea>span.score>em.num"
    ratingnum_element = "ul.placelist>li.PlaceItem.clickArea>div.rating.clickArea>span.score>a.numberofscore"
    re_element = "ul.placelist>li.PlaceItem.clickArea>div.rating.clickArea>a.review>em"
    add_element = "ul.placelist>li.PlaceItem.clickArea>div.info_item>div.addr"
    
    for x in driver.find_elements(By.CSS_SELECTOR, title_element) :
        title.append(x.text)
    
    for x in driver.find_elements(By.CSS_SELECTOR, cate_element) :
        category.append(x.text)
    
    for x in driver.find_elements(By.CSS_SELECTOR, rating_element) :
        rating.append(x.text)    
        
    for x in driver.find_elements(By.CSS_SELECTOR, ratingnum_element) :
        rating_num.append(x.text)
        
    for x in driver.find_elements(By.CSS_SELECTOR, re_element) :
        review.append(x.text)
        
    for x in driver.find_elements(By.CSS_SELECTOR, add_element) :
        address.append(x.text)
        
df = pd.DataFrame({'Title' : title, 'Category' : category, 
                   'Rating' : rating, 'RatingNum' : rating_num, 'Review' : review, 'Address' : address})


In [23]:
len(title), len(category), len(rating), len(rating_num), len(review), len(address)

(75, 75, 75, 75, 75, 75)

In [24]:
df = pd.DataFrame({'Title' : title, 'Category' : category, 
                   'Rating' : rating, 'RatingNum' : rating_num, 'Review' : review, 'Address' : address})

In [25]:
df.shape

(75, 6)

In [27]:
new_add = []
for x in df['Address'] :
    new_add.append(x.split('\n')[0])
df['NewAddress'] = new_add
df.head()

,Title,Category,Rating,RatingNum,Review,Address,NewAddress
0,A 명동교자 본점,분식,3.6,912건,"2,200",서울 중구 명동10길 29\n(지번) 명동2가 25-36,서울 중구 명동10길 29
1,B 진미평양냉면,냉면,3.6,280건,819,서울 강남구 학동로 305-3 정각빌딩 1층\n(지번) 논현동 115-10,서울 강남구 학동로 305-3 정각빌딩 1층
2,C 금돼지식당,"육류,고기",3.8,247건,"1,095",서울 중구 다산로 149\n(지번) 신당동 370-69,서울 중구 다산로 149
3,D 어니언 안국점,카페,3.5,215건,"1,432",서울 종로구 계동길 5\n(지번) 계동 146-1,서울 종로구 계동길 5
4,E 중앙해장,해장국,3.9,404건,943,서울 강남구 영동대로86길 17 육인빌딩 1층\n(지번) 대치동 996-16,서울 강남구 영동대로86길 17 육인빌딩 1층


In [ ]:
ㅇㄹ.